In [1]:
import sys
import re
import copy
from tqdm import tqdm
import random
import json

import time
from tqdm import tqdm
import requests
import faiss
import numpy as np
from SimCSE_append import SimCSE_append as SimCSE
from shared.prompt import instance
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline
from KNN_RE import get_train_example, find_knn_example, find_lmknn_example
from load_model import get_emb,Args
from label_explain import SemEval_label2exp
sys.path.append("/group/40064/johnbli/Code/UGC")
from examples.gpt_http import retry_request_openai_summary
from distort import Filter

params = {
    # 任务信息
    "env": "production",  # 必填，（取值范围：production、development）
    "task_name": "逃出精神病院",
    "task_type": "文本生成任务",  # 必填，任务类型（取值范围：qa任务、文本生成任务、整段对话生产、多轮对话、其他）
    "task_description": "",
    "business_id":"134"
}
import copy
import math

/data/miniconda3/envs/env_clone-3.9.16/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/miniconda3/envs/env_clone-3.9.16/lib/python3.9/site-packages/torch_npu/utils/path_manager.py:82: UserWarning: Warning: The /usr/local/Ascend/ascend-toolkit/latest owner does not match the current user.
  warnings.warn(f"Warning: The {path} owner does not match the current user.")
/data/miniconda3/envs/env_clone-3.9.16/lib/python3.9/site-packages/torch_npu/utils/path_manager.py:82: UserWarning: Warning: The /usr/local/Ascend/ascend-toolkit/8.0.RC2.alpha003/x86_64-linux/ascend_toolkit_install.info owner does not match the current user.
  warnings.warn(f"Warning: The {path} owner does not match the current user.")


In [2]:
dataset_path = "/group/40064/johnbli/Code/GPT-RE/dataset/semeval_gpt/test.json"

test_data = []
with open(dataset_path, "r") as f:
    for line in f:
        line = json.loads(line)
        test_data.append(line)


In [3]:
test_data[0]

{'sentences': [['the',
   'most',
   'common',
   'audits',
   'were',
   'about',
   'waste',
   'and',
   'recycling',
   '.']],
 'ner': [[[3, 3, 'sub'], [6, 6, 'obj']]],
 'predicted_ner': [[[3, 3, 'sub'], [6, 6, 'obj']]],
 'relations': [[[3, 3, 6, 6, 'Message-Topic']]],
 'doc_key': '0oftrain'}

In [4]:
semeval_idtoprompt = {0:"NONE",1:"CAUSE AND EFFECT", 2:"COMPONENT AND WHOLE", 3:"ENTITY AND DESTINATION",4:"ENTITY AND ORIGIN",5:"PRODUCT AND PRODUCER",6:"MEMBER AND COLLECTION",7:"MESSAGE AND TOPIC",8:"CONTENT AND CONTAINER",9:"INSTRUMENT AND AGENCY"}
semeval_reltoid = {"Other":0,"Cause-Effect":1, "Component-Whole":2, "Entity-Destination":3, "Entity-Origin":4, "Product-Producer": 5, "Member-Collection":6, "Message-Topic": 7, "Content-Container":8, "Instrument-Agency":9}


In [5]:
GPT_PROMPT = '''
You are a relation extraction expert specializing in the field of relation extraction, responsible for identifying and extracting the relationship between entities from text to support applications such as information retrieval, knowledge graph construction and data analysis. The following is the entity information:

Entity 1: {ent1}

Entity 2: {ent2}

The categories of relations include the following: {rels}

Your task is to process the text given below to accurately identify and extract the relationship between entity 1 and entity 2 in the text from the list of relation categories.

**Requirements**
You must accurately identify the type of relationship between entities in the sentence.
The response should not contain any information other than the entity category.

**Examples**
{examples}

Please judge the relationship between the following sentences based on the above examples:
**Input**
{sen}

Output format:
Relation: XXX

'''

In [6]:
LLaMA_PROMPT = '''
You are a system prompt generation expert specializing in the field of relation extraction, responsible for identifying and extracting the relationship between entities from text to support applications such as information retrieval, knowledge graph construction and data analysis. The following is the entity information:

Entity 1: {ent1}

Entity 2: {ent2}

The categories of relations include the following: {rels}

Your task is to process the text given below to accurately identify and extract the relationship between entity 1 and entity 2 in the text from the list of relation categories.

**Examples**
{examples}

**Requirements**
You must accurately identify the type of relationship between entities in the sentence
The response should not contain any information other than the entity category

Please judge the relationship between the following sentences based on the above examples:
**Input**
{sen}

Output format:
Relation: XXX

'''

**Qwen2.5-7B**

In [7]:
model_name_or_path = '/group/40064/heyfonli/model_hub/Qwen2.5-7B-Instruct/'
# model_name_or_path = "/group/40064/johnbli/LLM-based-models/LLama3/Meta-Llama-3-8B-Instruct"

In [8]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map='npu:2', trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 4/4 [00:32<00:00,  8.24s/it]


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def get_Qwen_res_7b(ent1, ent2, rels, examples, sentence):
    prompt = GPT_PROMPT.format(ent1 = ent1,ent2 = ent2,rels=rels,examples=examples, sen = sentence)
    messages = [{'role': 'system', 'content': 'You are an AI assistant '}, {'role': 'user', 'content': prompt}]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    # print(text)
    model_inputs = tokenizer([text], return_tensors="pt").to('npu:2')
    generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.9,
            top_p = 0,
            top_k = 1
        )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return [response]


In [11]:
TRAIN_SUP = '''
Sample {num}:
    Entity 1: {ent1}
    Entity 2: {ent2}
    Input:{sen}
    Output:Relation: {label}
'''

In [12]:
use_instance = True
use_knn = True
use_pipline = False
k_sample = 5

example_dict = get_train_example("/group/40064/johnbli/Code/GPT-RE/dataset/semeval_gpt/train.json", semeval_reltoid, 0)
train_list = [x for y in example_dict.values() for x in y]
# train_list = [x for x in train_list if semeval_reltoid[x["relations"][0][0][4]] != 0]
train_sentences = [instance(x).reference for x in train_list]
train_dict = {instance(x).reference:x for x in train_list}


In [13]:
if use_knn:
    knn_model = SimCSE("/group/40064/johnbli/bert-based-models/sup-simcse-roberta-large")
    knn_model.build_index(train_sentences, device="cpu")

/data/miniconda3/envs/env_clone-3.9.16/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
01/12/2025 11:37:32 - INFO - simcse.tool -   Encoding embeddings for sentences...
100%|██████████| 102/102 [02:01<00:00,  1.19s/it]
01/12/2025 11:39:34 - INFO - simcse.tool -   Building index...
01/12/2025 11:39:34 - INFO - simcse.tool -   Use CPU-version faiss
01/12/2025 11:39:34 - INFO - simcse.tool -   Finished


In [14]:
import torch
from distort import Distort
from distort import Filter
if use_pipline:

    # res = faiss.StandardGpuResources()  # 不再需要初始化GPU资
    # gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)  # 不需要转移到GPU
    index_flat = faiss.IndexFlatL2(768)  # 创建一个使用L2距离的平面索引
    cpu_index_flat = index_flat  # 直接使用CPU索引
    args =Args()
    dataset = 'SemEval'
    retriever_plm = "/group/40064/johnbli/bert-based-models/bert_base_uncased"
    device = torch.device("npu:2")
    train_samples = get_emb(args, train_list, dataset, retriever_plm, k_sample, device) #使用全部数据集
    # train_samples = get_emb(args, "", dataset, retriever_plm, k_sample, device) #使用部分数据集

    embed_list = np.array(train_samples)
    cpu_index_flat.add(embed_list)  # 使用CPU索引添加特征向量


In [15]:
def get_example(item):
    sentences_list = item["sentences"][0]
    ent1 = " ".join(sentences_list[item["ner"][0][0][0]:item["ner"][0][0][1]+1])
    ent2 = " ".join(sentences_list[item["ner"][0][1][0]:item["ner"][0][1][1]+1])
    sentence = " ".join(sentences_list)
    label = item["relations"][0][0][-1]
    return ent1,ent2,sentence,label

**测试代码-top5**

In [85]:
def get_embeddings(items, model, is_train = True, delta = 0.1):
    examples_mask = [instance(x).reference for x in items]
    examples = [instance(x['origin']).reference for x in items]
    embeddings_mask = model.encode(examples_mask)
    embedding = model.encode(examples)

    # 去除最关键部分
    model.encode_mask(embedding, examples, k=2)
    # 修改状态向量
    sub_embedding = embedding - delta * embeddings_mask
    if not is_train:
        return embedding
    OUT = torch.cat([embedding, sub_embedding], dim = 0)
    return OUT


In [86]:
from KNN_RE import check_list, check_Semantics_list, get_top_index

if use_knn:
    knn_select = find_knn_example(knn_model, test_data, train_dict, k_sample, True)
    Filter_all = Filter(train_list)
    # knn_select = check_list_top5(knn_select, Filter_all)
    # # 创建语义反事实空间
    semantic_items = Filter_all.entMask_distort()
    train_distort = [Filter_all.get_distort_sen(x) for x in semantic_items]
    train_distor_dict = {Filter_all.get_distort_sen(x):x for x in semantic_items}
    # # 创建长度反事实空间
    # lengths_items = Filter_all.length_distort('/group/40064/johnbli/Code/GPT-RE/dataset/semeval_gpt/train_len_distort.json')
    # train_distort = [instance(x).reference for x in lengths_items]
    # train_distor_dict = {instance(x).reference : x for x in lengths_items}
    # # 创建分布反事实空间
    # knn_model_distort.build_index(train_distort, device="cpu")


    knn_model_distort = SimCSE("/group/40064/johnbli/bert-based-models/sup-simcse-roberta-large")
    index_flat = faiss.IndexFlatL2(1024)  # 创建一个使用L2距离的平面索引
    cpu_index_flat = index_flat  # 直接使用CPU索引
    train_embeddings = get_embeddings(semantic_items, knn_model_distort)
    embed_list = np.array(train_embeddings)
    cpu_index_flat.add(embed_list)  # 使用CPU索引添加特征向量
    distribution_score = check_Semantics_list(knn_select, Filter_all) #分布偏差分数

.9.16/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/data/miniconda3/envs/env_clone-3.9.16/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/data/miniconda3/envs/env_clone-3.9.16/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/data/miniconda3/envs/env_clone-3.9.16/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of 

In [98]:
if use_knn:
    top_index = get_top_index(knn_select, distribution_score, 0.07)
    # 修改examples
    new_test_data = []
    for idx in top_index:
        new_test_data.append(test_data[idx])
    

In [99]:
def entMask_distort(items):
    OUT = []
    for item_train in items:
        item = copy.deepcopy(item_train)
        sentences_list = item["sentences"][0]
        ent1 = 'Mask1'
        ent2 = 'Mask2'
        if item["ner"][0][1][0] < item["ner"][0][0][0]:
            sentences_list[item["ner"][0][0][0]:item["ner"][0][0][1]+1] = [ent1]
            sentences_list[item["ner"][0][1][0]:item["ner"][0][1][1]+1] = [ent2]
        else:
            sentences_list[item["ner"][0][1][0]:item["ner"][0][1][1]+1] = [ent2]
            sentences_list[item["ner"][0][0][0]:item["ner"][0][0][1]+1] = [ent1]

        item["sentences"][0] = sentences_list
        p1 = -1
        p2 = -1
        for i,word in enumerate(sentences_list):
            if word == 'Mask1':
                p1 = i
            if word == 'Mask2':
                p2 = i
        if p1 != -1 and p2 != -1:
            item['origin'] = item_train
            item["ner"][0][0][0] = p1 
            item["ner"][0][0][1] = p1
            item["ner"][0][1][0] = p2 
            item["ner"][0][1][1] = p2
            item['relations'] = [[[p1, p1, p2, p2, item["relations"][0][0][-1]]]]
            OUT.append(item)
    return OUT

In [100]:
new_test_data = entMask_distort(new_test_data)
test_samples = get_embeddings(new_test_data, knn_model_distort, False, 0)
D, I = cpu_index_flat.search(test_samples, 3*k_sample)
lm_select = []
train_len = len(train_sentences)
# 保证没有之前的KNN选的，也没有重复的
for j in range(I.shape[0]):
    origin_select = knn_select[top_index[j]]
    index_list = []
    for i in I[j,:3*k_sample]:
        index = i % train_len
        if (index not in index_list):# and (train_dict[train_sentences[index]] not in origin_select):
            index_list.append(index)
        if len(index_list)>=k_sample:
            break
    j_select = [train_dict[train_sentences[t]] for t in index_list]
    lm_select.append(j_select)

100%|██████████| 3/3 [00:02<00:00,  1.06it/s]


In [101]:
for i, idx in enumerate(top_index):
    # a = [x['origin'] for x in new_items[i]]
    knn_select[idx] = lm_select[i]


In [102]:
if use_pipline:
    test_samples = get_emb(args, test_data, dataset, retriever_plm, k_sample, device)
    D, I = cpu_index_flat.search(test_samples, k_sample)
    lm_select = []
    for j in range(I.shape[0]):
        j_select = [train_dict[train_sentences[i]] for i in I[j,:k_sample]]
        lm_select.append(j_select)
    # lm_select = find_lmknn_example(cpu_index_flat, test_data, train_dict, train_sentences, k_sample)

In [109]:
Labels = []
predict = []
op = []
p1 = []
p2 = []
num = 0
ids = 0
predict1 = []
filters = Filter(train_list)
for test_item in tqdm(test_data):

    sentences_list = test_item["sentences"][0]
    ent1 = " ".join(sentences_list[test_item["ner"][0][0][0]:test_item["ner"][0][0][1]+1])
    ent2 = " ".join(sentences_list[test_item["ner"][0][1][0]:test_item["ner"][0][1][1]+1])
    sentence = " ".join(sentences_list)

    rels = ""
    rels_lst = {}
    for k,v in semeval_idtoprompt.items():
        rels = rels + v + '; '
        rels_lst[v] = k
    # 更详细的关系描述
    # for k,v in SemEval_label2exp.items():  
    #     rels = rels + k + ":" + v + '\n'
    label = semeval_reltoid[test_item["relations"][0][0][-1]]
    # if ids not in top_index:
    #     ids+=1
    #     continue
    
    # 添加instance
    example = ""
    if use_instance:
        if use_knn:
            selected_items = knn_select[ids]
            ids += 1
            # print(selected_items)
        elif use_pipline:
            selected_items = lm_select[ids]
            ids += 1
        else:
            current_list, wrong_list = filters.frequency_filter(test_item,10)
            selected_items = random.sample(wrong_list, k_sample)
        for i,item in enumerate(selected_items):

            train_ent1,train_ent2,train_sentence,train_label = get_example(item)
            train_label = semeval_idtoprompt[semeval_reltoid[train_label]]
            example = example + TRAIN_SUP.format(num = i, ent1 = train_ent1,ent2 = train_ent2,label=train_label, sen = train_sentence)
        # print(example)

    Labels.append(label)
    # res_lst = get_Qwen_res(ent1, ent2, rels, sentence)    #Qwen2.5-70b
    # res_lst = gen_GPT_first_chat(ent1, ent2, rels, example, sentence)     #GPT-70b
    res_lst = get_Qwen_res_7b(ent1, ent2, rels, example, sentence)        #Qwen2.5-7b
     
    res = res_lst[0].replace("Relation: ","") # 无理由

    if res not in rels_lst:
        num += 1
        predict.append(0 if label!=0 else 1)
    else:
        predict.append(rels_lst[res])


100%|██████████| 2717/2717 [17:18<00:00,  2.62it/s]


In [110]:
from sklearn.metrics import f1_score

# 计算 F1 值
macro_f1 = f1_score(Labels, predict, average='macro')  # 'weighted' 考虑各类标签的不平衡问题
micro_f1 = f1_score(Labels, predict, average='micro')  # 'weighted' 考虑各类标签的不平衡问题

print("micro_f1 Score:", micro_f1)
print("macro_f1 Score:", macro_f1)

micro_f1 Score: 0.6021347073978653
macro_f1 Score: 0.5989836677757282


In [105]:
lm_select[4]

[{'sentences': [['a',
    'little',
    'girl',
    'came',
    'out',
    'of',
    'the',
    'closet',
    ',',
    'her',
    'face',
    'swollen',
    'and',
    'red',
    'from',
    'crying',
    '.']],
  'ner': [[[2, 2, 'sub'], [7, 7, 'obj']]],
  'predicted_ner': [[[2, 2, 'sub'], [7, 7, 'obj']]],
  'relations': [[[2, 2, 7, 7, 'Entity-Origin']]],
  'doc_key': '4911oftrain'},
 {'sentences': [['it',
    'was',
    'intermittently',
    'wet',
    'and',
    'dry',
    ',',
    'and',
    'almost',
    'always',
    'followed',
    'with',
    'a',
    'whine',
    'and',
    'often',
    'a',
    'few',
    'tears',
    'from',
    'the',
    'terrible',
    'pain',
    '.']],
  'ner': [[[22, 22, 'obj'], [18, 18, 'sub']]],
  'predicted_ner': [[[22, 22, 'obj'], [18, 18, 'sub']]],
  'relations': [[[22, 22, 18, 18, 'Cause-Effect']]],
  'doc_key': '2318oftrain'},
 {'sentences': [['marital',
    'breakdown',
    'is',
    'often',
    'attributed',
    'to',
    'unresolved',
    'gr

In [106]:
new_test_data[4]

{'sentences': [['i',
   'ca',
   "n't",
   'help',
   'myself',
   'crying',
   ',',
   'especially',
   'when',
   'the',
   'Mask1',
   'went',
   'away',
   'from',
   'the',
   'Mask2',
   'in',
   'sadness',
   'during',
   'the',
   'boy',
   'was',
   'in',
   'hospital',
   '.']],
 'ner': [[[10, 10, 'sub'], [15, 15, 'obj']]],
 'predicted_ner': [[[10, 10, 'sub'], [15, 15, 'obj']]],
 'relations': [[[10, 10, 15, 15, 'Entity-Origin']]],
 'doc_key': '1917oftrain',
 'origin': {'sentences': [['i',
    'ca',
    "n't",
    'help',
    'myself',
    'crying',
    ',',
    'especially',
    'when',
    'the',
    'girl',
    'went',
    'away',
    'from',
    'the',
    'boy',
    'in',
    'sadness',
    'during',
    'the',
    'boy',
    'was',
    'in',
    'hospital',
    '.']],
  'ner': [[[10, 10, 'sub'], [15, 15, 'obj']]],
  'predicted_ner': [[[10, 10, 'sub'], [15, 15, 'obj']]],
  'relations': [[[10, 10, 15, 15, 'Entity-Origin']]],
  'doc_key': '1917oftrain'}}

In [107]:
test_data[38]

{'sentences': [['the',
   'movie',
   'featured',
   'a',
   'mutated',
   'sheep',
   'as',
   'the',
   'titular',
   'monster',
   ',',
   'which',
   'grew',
   'larger',
   'throughout',
   'the',
   'movie',
   ',',
   'until',
   'it',
   'exploded',
   'in',
   'the',
   'climactic',
   'scene',
   '.']],
 'ner': [[[1, 1, 'sub'], [5, 5, 'obj']]],
 'predicted_ner': [[[1, 1, 'sub'], [5, 5, 'obj']]],
 'relations': [[[1, 1, 5, 5, 'Message-Topic']]],
 'doc_key': '38oftrain'}

In [108]:
predict[6]

7

In [ ]:
micro_f1 Score: 0.6105263157894737
macro_f1 Score: 0.5661905034097093